Expected Stock Move = Stock Price x (Implied Volatility/100) x square root (days to expiration/252 trading days)

Implied volatility can then be derived from the cost of the option. In fact, if there were no options traded on a given stock, there would be no way to calculate implied volatility.

In [1]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
import datetime
from datetime import datetime as dt, timedelta
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#defining a normal distribution
N = norm.cdf

#call price function, sigma=volatility(σ)
def BSM_call_price(S, K , r , T , sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma * np.sqrt(T)) 
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2) 

#put price function, sigma = volatility(σ)
def BSM_put_price(S, K , T , r , sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * N(-d2) - S * N(-d1)

#S = current stock price
#K = strike
#r = risk-free interest
#T = time to expiration
#sigma = volatility 

In [3]:
def imp_vol(S, K , r , T ,market_price, option_type = 'C' or 'P'):
        price_difference = 0.001
        volatility = 0.2
        step = 0.001
      
        for i in range(1000): 
            if option_type == 'C':
                price = BSM_call_price(S, K, r, T, sigma = volatility)    
            else: 
                price = BSM_put_price(S, K , T , r , sigma = volatility)
    
            difference = market_price - price
            if difference > price_difference:
                  volatility = volatility + step
            elif difference < 0 and abs(difference) > price_difference:
                  volatility = volatility - step
            elif abs(difference) < price_difference:
                  return volatility
        return volatility #if best value not found

In [9]:
def expected_move_IV(stock, K, r, T, market_price, option_type = 'P' or 'C'):
    #first we are calculating the number of days left until the option expiration
    future = datetime.datetime.strptime(T, "%Y-%m-%d")
    today = datetime.datetime.today()
    delta = (future - today).days
    t = delta/365

    #here we are downloading the stock's current price 
    ticker = yf.Ticker(stock)
    data = ticker.history()
    last_quote = data['Close'].iloc[-1]

    #calculating the implied volatility 
    ivc = imp_vol(last_quote, K, r, t, market_price, option_type)
    
    print("implied vol - Close", ivc)
    
    #calculating the ijmplied volatilty using the open price
    last_quote = data['Open'].iloc[-1]

    #calculating the implied volatility 
    ivo = imp_vol(last_quote, K, r, t, market_price, option_type)
    
    print("implied vol - Open", ivo)

    #calculating the expected move using implied volatility 
    move = last_quote * (ivc) * np.sqrt(t)

    #calculating the upper and lower bounds of the expected move 
    upper_bound = last_quote + round(move,2)
    lower_bound = last_quote - round(move,2)

    #adding historical dates for a chart

    backdate = dt.now() + timedelta(days=-100)
    date = backdate.strftime("%Y-%m-%d")

    #downloading stock data 

    frame = yf.download(stock, start = date)
    #adding the future dates into the data dataframe
    b = pd.date_range(start = date, end = future)
    frame.index.append(b)

    #graphing historical performance and the expected price range

    fig = px.line(frame, x=frame.index, y=frame['Close'], title = f'{stock} Expected Price Range (+/- {round(move,2)})')
    fig.update_xaxes(range=[date, future])
    fig.add_hline(y=upper_bound, line = dict(color = 'red', width =3), line_dash="dot", annotation_text=upper_bound,
              annotation_position="bottom right")
    fig.add_hline(y=lower_bound, line = dict(color = 'red', width =3), line_dash="dot", annotation_text=lower_bound, 
              annotation_position="bottom right")
    fig.update_yaxes(title_text="Stock Price")
    fig.update_xaxes(title_text="Date")
    fig.update_layout(height=700, width=1500, 
                  showlegend=False)

    fig.add_trace(
    go.Scatter(
        x=frame.index,
        y= frame['Open'], 
        name='Open Price',
        mode='lines',
        showlegend=True,
        opacity = 0.5,
        marker=dict(
        color='orange',
        size=5,
        opacity=0.3,
        line=dict(
            color='MediumPurple',
            width=2)
        )
            )
        )

    return fig.show()

In-the-Money (ITM) Options: These options have a strike price that is favorable compared to the current market price of the underlying asset. ITM options are often more sensitive to changes in implied volatility and are commonly used for calculating implied volatility.

At-the-Money (ATM) Options: ATM options have a strike price close to the current market price of the underlying asset. They are also widely used in implied volatility calculations, as they are typically the most liquid and frequently traded options.

Out-of-the-Money (OTM) Options: These options have a strike price less favorable than the current market price of the underlying asset. OTM options are less commonly used for implied volatility calculations but can provide information about market sentiment and expectations.

In [7]:
#ITM - 2023 - 11-17
print("In the Money:")
print("Put:")
expected_move_IV('SPY', 440, 0.0477, '2023-11-17', (6.43+6.46)/2, 'P')
print("Call:")
expected_move_IV('SPY', 430, 0.0477, '2023-11-17', (8.73+8.78)/2, 'C')
print("At the Money")
print("Put:")
expected_move_IV('SPY', 436, 0.0477, '2023-11-17', (4.28+4.29)/2, 'P')
print("Call:")
expected_move_IV('SPY', 435, 0.0477, '2023-11-17', (5.29+5.30)/2, 'C')


In the Money:
Put:
implied vol - Close 0.13599999999999995
implied vol - Open 0.06399999999999989
[*********************100%***********************]  1 of 1 completed


Call:
implied vol - Close 0.14799999999999996
implied vol - Open 0.20400000000000001
[*********************100%***********************]  1 of 1 completed


At the Money
Put:
implied vol - Close 0.13799999999999996
implied vol - Open 0.09599999999999992
[*********************100%***********************]  1 of 1 completed


Call:
implied vol - Close 0.13799999999999996
implied vol - Open 0.178
[*********************100%***********************]  1 of 1 completed


In [10]:
print("Put:")
expected_move_IV('SPY', 440, 0.0477, '2023-11-17', (6.43+6.46)/2, 'P')

Put:
implied vol - Close 0.13599999999999995
implied vol - Open 0.06399999999999989
[*********************100%***********************]  1 of 1 completed


Ref: https://medium.datadriveninvestor.com/calculating-expected-stock-move-using-implied-volatility-in-python-6223e947cfc0

In [11]:
#correlation between open price and close price


backdate = dt.now() + timedelta(days=-252)
date = backdate.strftime("%Y-%m-%d")

stock = "QQQ"
frame = yf.download(stock, start = date)



frame['Close'].corr(frame['Open'])

[*********************100%***********************]  1 of 1 completed


0.9927635029691967

In [42]:
frame.describe()

In [45]:
frame